# Phase 3 Validation: Walk-Forward and CPCV Orchestrator

This notebook is used to validate the implementation of the Walk-Forward and CPCV orchestrators.

In [ ]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from backtesting.orchestrator import Orchestrator
from backtesting.strategies.sma_crossover import SmaCrossover
from backtesting.portfolio import Portfolio
from backtesting.execution import Execution, FlatCommission, GaussianSlippage
from backtesting.performance import Performance
from data_handler import DataHandler
from data_storage.storage_backend import HybridStorageManager

## 1. Walk-Forward Analysis

In [ ]:
# Create a sample dataframe
data = pd.DataFrame({
    "asset": ["AAPL"] * 1000,
    "close": [100 + i + (i % 20) * 5 - (i % 30) * 2 for i in range(1000)],
})
data['date'] = pd.to_datetime(pd.date_range(start='2020-01-01', periods=1000))
data = data.set_index('date')

# Create the orchestrator
storage_manager = HybridStorageManager({})
data_handler = DataHandler(storage_manager)
orchestrator = Orchestrator(
    data_handler=data_handler,
    strategy_cls=SmaCrossover,
    portfolio_cls=Portfolio,
    execution_cls=lambda: Execution(
        commission_model=FlatCommission(0.001),
        slippage_model=GaussianSlippage(0, 0.001),
    ),
    performance_cls=Performance,
)

# Run a walk-forward backtest
walk_forward_config = {
    "run_id": "walk_forward_validation",
    "walk_forward": {
        "train_period": 200,
        "test_period": 100,
        "step_size": 100,
    },
    "strategy_params": {"short_window": 20, "long_window": 50},
    "portfolio_params": {"initial_cash": 100000},
}

orchestrator.run_ray(walk_forward_config, data)
orchestrator.to_json("walk_forward_validation.json")

with open("walk_forward_validation.json") as f:
    wfa_results = json.load(f)


In [ ]:
def plot_wfa_equity_curves(results):
    fig = go.Figure()
    for slice in results['slices']:
        equity_curve = slice['metrics']['equity_curve']
        fig.add_trace(go.Scatter(x=list(range(len(equity_curve))), y=equity_curve, mode='lines', name=f"Slice {slice['slice_id']}"))
    fig.update_layout(title="Walk-Forward Equity Curves", xaxis_title="Time", yaxis_title="Equity")
    fig.show()

In [ ]:
plot_wfa_equity_curves(wfa_results)

## 2. Combinatorial Purged Cross-Validation (CPCV)

In [ ]:
# Run a CPCV backtest
cpcv_config = {
    "run_id": "cpcv_validation",
    "cpcv": {
        "N": 10,
        "k": 2,
        "embargo_pct": 0.05,
    },
    "strategy_params": {"short_window": 20, "long_window": 50},
    "portfolio_params": {"initial_cash": 100000},
}

orchestrator.run_ray(cpcv_config, data)
orchestrator.to_json("cpcv_validation.json")

with open("cpcv_validation.json") as f:
    cpcv_results = json.load(f)

In [ ]:
def plot_cpcv_sharpe_distribution(results):
    sharpe_ratios = [s['metrics']['sharpe_ratio'] for s in results['slices']]
    fig = px.histogram(x=sharpe_ratios, nbins=20, title="CPCV Sharpe Ratio Distribution")
    fig.update_layout(xaxis_title="Sharpe Ratio", yaxis_title="Frequency")
    fig.show()

In [ ]:
plot_cpcv_sharpe_distribution(cpcv_results)